#BERT Model with FAISS Evaluation

In [1]:
import pandas as pd
import re
import string
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import tensorflow
import sklearn
from tqdm import tqdm
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Transformer model

In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.9 MB/s eta 0:00:00


## BERT model and tokenizer

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 512

In [7]:
# checking the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


### Loading model for evaluation

In [8]:
loaded_model = BertForSequenceClassification.from_pretrained('Saved_BERT_model')

In [9]:
loaded_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## FAISS

In [10]:
!pip install faiss-cpu  # for CPU version
# or
!pip install faiss-gpu  # for GPU version


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.0 MB/s eta 0:00:00


In [11]:
import faiss
import numpy as np

In [12]:
question_embeddings = np.load('/content/drive/Shareddrives/SEM 3/DATA298A  /Code/question_embeddings.npy')

In [13]:
# Convert question embeddings to a numpy matrix
question_embeddings_matrix = np.vstack(question_embeddings)

# Get the dimension of the embeddings
dimension = question_embeddings_matrix.shape[1]

# Create a FAISS index
index = faiss.IndexFlatL2(dimension)

# Add the question embeddings to the index
index.add(question_embeddings_matrix)


In [14]:
question_embeddings_matrix.shape

(20000, 768)

In [14]:

def encode(text, tokenizer, model, device):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt').to(device)
    with torch.no_grad():
        output = model.bert(**inputs)
    return output.last_hidden_state[:, 0, :].cpu().numpy()

In [15]:
def find_top_similar_question_indices(user_input, tokenizer, model, device, index, k=5):
    # Encode the user input
    user_input_embedding = encode(user_input, tokenizer, model, device)

    # Search the index for the top k similar embeddings
    _, indices = index.search(user_input_embedding, k)

    # Flatten the indices array
    indices = indices.flatten()

    return indices


In [16]:
### cleaning functions

def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_punctuation_and_lower(text):
    text = text.lower()
    return text.translate(str.maketrans(' ', ' ', string.punctuation))

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    return [word for word in word_tokens if word not in stop_words]

def remove_usernames(text: str) -> str:
    """
    Remove usernames starting with '@' from the given text.

    Args:
        text (str): The input text containing usernames.

    Returns:
        str: The text with usernames removed.
    """
    cleaned_text = re.sub(r"@[A-Za-z0-9_]+", "", text)
    return cleaned_text

## Groundtruth Ranking

In [17]:

ground_truth_df = pd.read_csv('Groundtruth_relevant_answers.csv')
ground_truth_df

,Question,Relevant_Answer,question_id
0,InfoBar Component not visible after rendering,<p>You don't appear to be returning anything f...,73279519
1,Writing bits (not hex values) to file in Linux,"<p>perl to the rescue:</p>\n<pre class=""lang-b...",73414802
2,Change value of outerscope variable inside a f...,<p>Your variables are mutated from inside of a...,73370188
3,SmartContract/Solidity: The transaction has be...,<p><code>attributes[tokenId].lastClaimDate</co...,73364556
4,This name error keeps popping up despite me im...,<p>Conn is not defined here. Either change <co...,73417861
...,...,...,...
19995,Using Mean.io - How to disable JS files aggreg...,<p>I am using the version 0.6.16. You can set ...,25698549
19996,CoreGraphics prevents frame modifications on o...,<p>Your question doesn't really have anything ...,25839033
19997,finding the average of the largest numbers inp...,<p>Using a <code>break;</code> instruction is ...,25815186
19998,Show Greek Date in English Format,<p>You can try using <code>[$-F800]</code> for...,25694275


In [18]:
# taking 2000 random questions

twothousand_q = ground_truth_df.sample(2000, random_state=11)
twothousand_q

,Question,Relevant_Answer,question_id
2440,cumsum in numpy ndarray (edited),<p>The solution of @Ben.T juste above worked p...,62200330
7382,Should I always provide default value in php f...,<p>MySQL does not support having default value...,24361295
13064,How to use/install Command 'host' Docker Alpin...,<p>Alpine has a website called pkgs.alpinelinu...,66851224
18810,Lua String patterns - shorter code,<p>You can at least chain and line-wrap it:</p...,15137004
359,Trouble removing package from Visual Studio 2010,"<p>Here's a <a href=""http://docs.nuget.org/doc...",11960360
...,...,...,...
9714,How to display a text in android after a butto...,"<p>OK, fjust write onClick method in OnClickLi...",9123887
13991,Terraform - How do I reference my Output insid...,<p>The module name is <code>subnetwork</code> ...,70510621
3128,Need help parsing TypeScript syntax,"<p>It is a lot to unpack, here it is line by l...",62851642
11191,GameObject store its own location in a tile ba...,<p>I'd advise you to store the location in the...,16743466


In [19]:
# call big query for scores and answer bodies

all_answersscore_df = pd.read_csv('All_answers_scores.csv')
all_answersscore_df

,question_id,question_title,answer_id,answer_body,score
0,13128,How can I combine several C/C++ libraries into...,27676078,"<p>On Linux, <a href=""https://en.wikipedia.org...",8
1,13128,How can I combine several C/C++ libraries into...,13136,<p>I'm not sure how to physically combine them...,0
2,13128,How can I combine several C/C++ libraries into...,22041,"<p>Maybe I'm misunderstanding, but don't you o...",0
3,13128,How can I combine several C/C++ libraries into...,13140,"<p>On Unix like systems, the ld and ar utiliti...",9
4,13128,How can I combine several C/C++ libraries into...,13206,<p>You could extract the object files from eac...,26
...,...,...,...,...,...
33208,73831805,Why are methods not valid constructors?,73832791,<p>As pointed out in the comments on the origi...,2
33209,73834279,No ability to publish newer build for external...,73834304,<p>I believe that there is possible to do more...,0
33210,73837100,How can I run multiple expressions inside of a...,73837248,<p>You're getting a syntax error due to the as...,-1
33211,73838452,Pad the missing data with NA for two matrices,73838548,<p>You can use <code>match</code> to find the ...,1


In [20]:
eval_q_list = twothousand_q['Question'].tolist()

In [21]:
def find_top_similar_answer_indices(user_input, tokenizer, model, device, index, k=5):
    # Encode the user input
    user_input_embedding = encode(user_input, tokenizer, model, device)

    # Search the index for the top k similar embeddings
    _, indices = index.search(user_input_embedding, k)

    # Flatten the indices array
    indices = indices.flatten()

    return indices

In [22]:
from sklearn.metrics import ndcg_score

In [26]:
### creating function for eval

def ndcg_evaluation(question_list):

  counter = 1
  for user_question in question_list:
    print(counter)
    counter += 1
    ndcg_total = []


    ## getting the top 5 indices - questions
    top_question_indices = find_top_similar_question_indices(user_question, tokenizer, loaded_model, device, index)

    #### Groundtruth ranks
    ## initiate score dataframe

    scores_df = pd.DataFrame({'score':[], 'answer_id':[], 'answer_body' : []})
    scores_df['answer_id'] = scores_df['answer_id'].astype(int)

    ## get scores
    for ind in top_question_indices:
      if len(scores_df) <= 5:
        q_id = ground_truth_df.loc[ind, 'question_id']
        filtered_df = all_answersscore_df[all_answersscore_df['question_id'] == q_id]
        # Select 'score','answr body' and 'answer_id' columns
        filtered_df = filtered_df[['score', 'answer_id','answer_body']]
        scores_df = pd.concat([scores_df,filtered_df], axis=0, ignore_index=True)
      else:
        break

    # Sort the scores DataFrame based on 'score' in descending order
    sorted_df = scores_df.sort_values(by='score', ascending=False)
    result_df = sorted_df.head(5)
    result_df.reset_index(drop=True, inplace=True)
    result_df = result_df.copy()
    result_df.loc[:, 'gt_relevance'] = list(range(len(result_df)-1, -1, -1))

    answer_ids = result_df['answer_id'].to_list()

    ## getting answer bodies
    answers_text_list = result_df['answer_body'].to_list()
    answers_embeddings = [encode(answer, tokenizer, loaded_model,device) for answer in answers_text_list]

    ## FAISS
    # Convert answer embeddings to a numpy matrix
    answer_embeddings_matrix = np.vstack(answers_embeddings)
    # Get the dimension of the embeddings
    dimension = answer_embeddings_matrix.shape[1]
    # Create a FAISS index
    answer_index = faiss.IndexFlatL2(dimension)
    # Add the question embeddings to the index
    answer_index.add(answer_embeddings_matrix)

    ## getting top answer indices
    top_answer_indices = find_top_similar_answer_indices(user_question, tokenizer, loaded_model, device, answer_index)

    result_df['top_indices'] = top_answer_indices

    fetched_values = []
    for i in top_answer_indices:
      test_rel_score = result_df['gt_relevance'].iloc[i]
      fetched_values.append(test_rel_score)
    result_df['test_relevance'] = np.array(fetched_values)


    ndcg = ndcg_score([result_df['gt_relevance'].to_list()], [result_df['test_relevance'].to_list()])

    ndcg_total.append(ndcg)

  ndcg_mean = np.mean(ndcg_total)

  return round(ndcg_mean, 4)


In [ ]:
# ndcg = ndcg_score([ndcg_df['gt_rank'].to_list()], [ndcg_df['rank'].to_list()])

# round(ndcg, 3)

# np.mean([ndcg])

BERT model - FAISS Evaluation

In [27]:
overall_ndgc_score = []

In [28]:
## 0 - 500
bert_ndcg_score1 = ndcg_evaluation(eval_q_list[:500])
overall_ndgc_score.append(bert_ndcg_score1)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [29]:
## 500 - 1000
bert_ndcg_score2 = ndcg_evaluation(eval_q_list[500:1000])
overall_ndgc_score.append(bert_ndcg_score2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [30]:
## 1000 - 1500
bert_ndcg_score3 = ndcg_evaluation(eval_q_list[1000:1500])
overall_ndgc_score.append(bert_ndcg_score3)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [31]:
## 1500 - 2000
bert_ndcg_score4 = ndcg_evaluation(eval_q_list[1500:2000])
overall_ndgc_score.append(bert_ndcg_score4)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [32]:
overall_ndgc_score

[0.8676, 0.8676, 0.8676, 0.8676]

In [33]:
print(f'The NDGC score of BERT - FAISS model (for 2000 test questions) is {np.mean(overall_ndgc_score):.4f}')

The NDGC score of BERT - FAISS model (for 2000 test questions) is 0.8676
